### Updating The FX_derivatives_all database 
Update with the TJ06 and TX06sap reports.

In [35]:
#Imports the daily TJ06 & TX06 and updates FX_Derivatives_All Database 
#which is located in the Financial_Database directory located within the Treasury drive

import pandas as pd
import csv
import datetime


#Putting into a variable yesterday's date to be used with the file names
today = datetime.date.today()
prevday = today - datetime.timedelta(days=1)
prevday = prevday.strftime('%m%d%Y')

dir_path = r'\\or1isi1vl3921nfs\sapnfsc\PRD003\0256'
file1_name = '\TX06_Extract' + prevday + '.txt'
file2_name = '\TJ06_Extract' + prevday + '.txt'
file_path1 = dir_path + file1_name
file_path2 = dir_path + file2_name


df1 = pd.read_csv(file_path1, delimiter = '|', skiprows=[0,2], header =[0])
df2 = pd.read_csv(file_path2, delimiter = '|', header =[0])

#df = pd.read_csv(r'\\or1isi1vl3921nfs\sapnfsc\PRD003\0256\TX06_Extract09212020.txt',

In [6]:
#let's rename our column headers without spaces so that we can then separate columns (currently 1 column) into many.

In [1]:
#df

In [36]:
df2

,Unnamed: 0,Type,Term Start,Due Date,Exercise,Settlement,Underlying,Strike,Transactn,Counterparty,Premium Amt,Premium Curr
NaN,Put,03/10/2020,10/01/2020,Europ.,Cash,"EUR 30,000,000.00-",1.1382,7868.0,2101.0,"788,700.00",USD,NaN
NaN,Put,03/10/2020,10/01/2020,Europ.,Cash,"GBP 17,000,000.00-",1.2925,7869.0,2101.0,"578,850.00",USD,NaN
NaN,Put,03/10/2020,10/01/2020,Europ.,Cash,"JPY 1,700,000,000-",103.5700,7870.0,2081.0,"550,800.00",USD,NaN
NaN,Put,03/24/2020,10/08/2020,Europ.,Cash,"EUR 30,000,000.00-",1.0865,7902.0,2.0,"909,013.00",USD,NaN
NaN,Put,04/07/2020,10/15/2020,Europ.,Cash,"EUR 32,000,000.00-",1.0950,7930.0,2121.0,"832,800.00",USD,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,Put,09/08/2020,04/08/2021,Europ.,Cash,"JPY 3,200,000,000-",105.5500,8251.0,2.0,"733,112.00",USD,NaN
NaN,Put,09/15/2020,04/15/2021,Europ.,Cash,"AUD 17,000,000.00-",0.7301,8264.0,2.0,"399,714.00",USD,NaN
NaN,Put,09/15/2020,04/15/2021,Europ.,Cash,"EUR 16,000,000.00-",1.1906,8265.0,2.0,"429,841.00",USD,NaN
NaN,Put,09/15/2020,04/15/2021,Europ.,Cash,"GBP 17,000,000.00-",1.2870,8266.0,12600.0,"752,846.87",USD,NaN
